## Running the Lomb-Scargle algorithm on an entire TESS dataset

In [2]:
# We recycle the function defined in the previous module for loading lightcurves:
from astropy.io import fits
from astropy.timeseries import LombScargle, BoxLeastSquares
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np


def load_LC(filename):
    hdul = fits.open(filename)
    t = hdul[1].data["TIME"]
    y = hdul[1].data["PDCSAP_FLUX"]
    dy = hdul[1].data["PDCSAP_FLUX_ERR"]

    TICID = hdul[1].header["TICID"]
    RA = hdul[1].header["RA_OBJ"]
    Dec = hdul[1].header["DEC_OBJ"]
    idx = y > 0
    t = t[idx]
    y = y[idx]
    dy = dy[idx]

    return t, y, dy, RA, Dec, TICID


def binning(phases, y, dy, N=100):
    binned_LC = []
    mean_phases = np.linspace(0, 1 - 1 / N, N)
    lightcurve = np.array((phases, y, dy)).T
    lightcurve = lightcurve[np.argsort(lightcurve[:, 0])]
    for i in mean_phases:
        lightcurve_bin = lightcurve[lightcurve[:, 0] > i]
        lightcurve_bin = lightcurve_bin[lightcurve_bin[:, 0] < i + 1 / N]
        weights = 1 / (lightcurve_bin[:, 2] ** 2)
        weighted_mean_flux = np.sum(lightcurve_bin[:, 1] * weights) / np.sum(weights)
        weighted_mean_flux_error = np.sqrt(1 / np.sum(weights))
        binned_LC.append((i + 0.5 / N, weighted_mean_flux, weighted_mean_flux_error))
    binned_LC = np.array(binned_LC)

    return binned_LC

In [2]:
# You are now ready to unleash this algorithm on the full TESS dataset. 
# As a test, let us check how long it actually takes the execute.

import time
import pathlib
from tqdm import tqdm
from multiprocessing.pool import ThreadPool


def process_file(f):
    '''
    Load data from the fits file and return the period-finding results from running Lomb-Scargle .
    '''
    t, y, dy, RA, Dec, TICID = load_LC(f)
    frequency, power = LombScargle(t, y, dy).autopower(maximum_frequency=360)
    best_period = 1 / frequency[np.argmax(power)]
    best_power = np.max(power)
    return TICID, RA, Dec, best_period, best_power


p = [str(pp) for pp in pathlib.Path("data/2min/").glob("*")]

t1 = time.perf_counter()
with ThreadPool(processes=16) as pool:
    r_LS = list(tqdm(pool.imap(process_file, p), total=len(p)))
    np.savetxt("LS.result", r_LS)
t2 = time.perf_counter()
print("Seconds to run: ", t2 - t1)

100%|█████████████████████████████████████████| 984/984 [00:14<00:00, 66.32it/s]

14.85924628599605


In [3]:
# Here is a slower algorithm designed to detect eclipses. 
# We'll probably want to use GPU based implementations of this one.
from astropy.timeseries import BoxLeastSquares

def process_file(f):

    t, y, dy, RA, Dec, TICID = load_LC(f)
    model = BoxLeastSquares(t * u.day, y, dy)
    periodogram = model.autopower(0.5)
    period_BLS = periodogram.period
    power_BLS = periodogram.power
    best_period = period_BLS[np.argmax(power_BLS)]
    best_power = np.max(power_BLS)
    return TICID, RA, Dec, best_period.value, best_power


p = [str(pp) for pp in pathlib.Path("data/2min/").glob("*")]

t1 = time.perf_counter()
with ThreadPool(processes=16) as pool:
    r_BLS = list(tqdm(pool.imap(process_file, p), total=len(p)))
    np.savetxt("BLS.result", r_BLS)
t2 = time.perf_counter()
print(t2 - t1)

100%|█████████████████████████████████████████| 984/984 [00:11<00:00, 82.14it/s]

11.999276863993146


In [5]:
r_BLS = np.array(r_BLS)
r_LS = np.array(r_LS)

for f in p:
    print(f)
    t, y, dy, RA, Dec, TICID = load_LC(f)
    solution_LS = r_LS[r_LS[:, 0] == TICID][0]
    solution_BLS = r_BLS[r_BLS[:, 0] == TICID][0]
    best_period_LS = solution_LS[3]
    best_power_LS = 1000 * solution_LS[4]
    phase = (t % best_period_LS) / best_period_LS
    binned_LC = binning(phase, y, dy, N=100)
    if best_power_LS > 10:
        plt.errorbar(binned_LC[:, 0], binned_LC[:, 1], binned_LC[:, 2], ls=" ")
        # you may want to save these files in /data/submit/{kerb_user} instead because you could sooner or later run into
        # your disk quota in your home directory
        plt.savefig(
            "output_LS/"
            + str(best_power_LS)
            + "_"
            + str(TICID)
            + "_"
            + str(RA)
            + "_"
            + str(Dec)
            + "_"
            + str(best_period_LS)
            + ".png"
        )
        plt.close()
        best_period_LS = solution_LS[3]
    best_period_BLS = solution_LS[3]
    best_power_BLS = 10 * solution_BLS[4]
    phase = (t % best_period_BLS) / best_period_BLS
    binned_LC = binning(phase, y, dy, N=100)
    if best_power_BLS > 10:
        plt.errorbar(binned_LC[:, 0], binned_LC[:, 1], binned_LC[:, 2], ls=" ")
        plt.savefig(
            "output_BLS/"
            + str(best_power_BLS)
            + "_"
            + str(TICID)
            + "_"
            + str(RA)
            + "_"
            + str(Dec)
            + "_"
            + str(best_period_BLS)
            + ".png"
        )
        plt.close()

2min/tess2021232031932-s0042-0000002052405591-0213-s_lc.fits
2min/tess2021232031932-s0042-0000000471014616-0213-s_lc.fits
2min/tess2021258175143-s0043-0000000620330854-0214-s_lc.fits
2min/tess2021258175143-s0043-0000000062178399-0214-s_lc.fits
2min/tess2021204101404-s0041-0000000471015226-0212-s_lc.fits
2min/tess2021175071901-s0040-0000001551010781-0211-s_lc.fits


/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))


2min/tess2021258175143-s0043-0000000640198164-0214-s_lc.fits
2min/tess2021175071901-s0040-0000001686956634-0211-s_lc.fits
2min/tess2021175071901-s0040-0000000743729769-0211-s_lc.fits
2min/tess2021232031932-s0042-0000000610916920-0213-s_lc.fits
2min/tess2021204101404-s0041-0000000950817374-0212-s_lc.fits
2min/tess2021175071901-s0040-0000001867758053-0211-s_lc.fits
2min/tess2021232031932-s0042-0000000471015156-0213-s_lc.fits


/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))
/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))


2min/tess2021204101404-s0041-0000001968530114-0212-s_lc.fits
2min/tess2021204101404-s0041-0000001102671080-0212-s_lc.fits
2min/tess2021232031932-s0042-0000000114058447-0213-s_lc.fits
2min/tess2021232031932-s0042-0000000471013908-0213-s_lc.fits
2min/tess2021204101404-s0041-0000000900691235-0212-s_lc.fits
2min/tess2021258175143-s0043-0000000630366283-0214-s_lc.fits
2min/tess2021175071901-s0040-0000000802816681-0211-s_lc.fits
2min/tess2021232031932-s0042-0000000402891559-0213-s_lc.fits
2min/tess2021232031932-s0042-0000002028884626-0213-s_lc.fits
2min/tess2021204101404-s0041-0000000027688847-0212-s_lc.fits
2min/tess2021232031932-s0042-0000000620065517-0213-s_lc.fits
2min/tess2021175071901-s0040-0000000158965515-0211-s_lc.fits
2min/tess2021175071901-s0040-0000001684596565-0211-s_lc.fits
2min/tess2021232031932-s0042-0000000640226759-0213-s_lc.fits
2min/tess2021204101404-s0041-0000000158013652-0212-s_lc.fits
2min/tess2021258175143-s0043-0000000438060759-0214-s_lc.fits
2min/tess2021232031932-s

...

/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))


2min/tess2021175071901-s0040-0000001847213698-0211-s_lc.fits
2min/tess2021232031932-s0042-0000002028913070-0213-s_lc.fits
2min/tess2021204101404-s0041-0000001001416172-0212-s_lc.fits
2min/tess2021175071901-s0040-0000000356268485-0211-s_lc.fits
2min/tess2021204101404-s0041-0000000900740806-0212-s_lc.fits


/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))


2min/tess2021258175143-s0043-0000000086543596-0214-s_lc.fits
2min/tess2021175071901-s0040-0000001920685932-0211-s_lc.fits
2min/tess2021175071901-s0040-0000001686601724-0211-s_lc.fits


/tmp/ipykernel_42508/1380486706.py:34: RuntimeWarning: invalid value encountered in double_scalars
  weighted_mean_flux=np.sum(lightcurve_bin[:,1]*weights)/np.sum(weights)
/tmp/ipykernel_42508/1380486706.py:35: RuntimeWarning: divide by zero encountered in double_scalars
  weighted_mean_flux_error=np.sqrt(1/np.sum(weights))
